In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
# pip install requests beautifulsoup4

In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:
url = "https://www.nseindia.com/api/equity-stockIndices?index=NIFTY%2050"

In [5]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept": "application/json",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.nseindia.com/"
}

In [6]:
session = requests.Session()
session.get("https://www.nseindia.com", headers=headers)  # warm-up request

<Response [200]>

In [7]:
response = session.get(url, headers=headers)
data = response.json()

In [8]:
nifty_price = data["data"][0]["lastPrice"]
print("Latest NIFTY 50 Price:", nifty_price)

Latest NIFTY 50 Price: 25048.65


In [9]:
def get_nifty_atm(price):
    if price%100 >= 50:
        return price//100 * 100 +100
    else:
        return price//100 * 100

In [10]:
nifty_price = get_nifty_atm(nifty_price)

In [30]:
df = pd.read_csv("option-chain-ED-NIFTY-27-Jan-2026 (3).csv",skiprows=1)

In [31]:
df.columns

Index(['Unnamed: 0', 'OI', 'CHNG IN OI', 'VOLUME', 'IV', 'LTP', 'CHNG',
       'BID QTY', 'BID', 'ASK', 'ASK QTY', 'STRIKE', 'BID QTY.1', 'BID.1',
       'ASK.1', 'ASK QTY.1', 'CHNG.1', 'LTP.1', 'IV.1', 'VOLUME.1',
       'CHNG IN OI.1', 'OI.1', 'Unnamed: 22'],
      dtype='object')

In [32]:
list_of_columns = ['OI','CHNG IN OI', 'STRIKE', 'CHNG IN OI.1','OI.1']

In [33]:
df = df[list_of_columns]

In [34]:
def clean(text):
    return float(text.replace(",","").replace("-","0"))

In [35]:
for i in df.columns:
    df[i] = df[i].apply(clean)

In [36]:
df = df.rename(columns={'OI':"call oi","OI.1":"put oi","STRIKE":"strike"})

In [37]:
df['pcr'] = df['put oi'] / df['call oi']

In [38]:
df = df[df['strike']%100==0]

In [39]:
df

,call oi,CHNG IN OI,strike,CHNG IN OI.1,put oi,pcr
0,0.0,0.0,23400.0,10985.0,48069.0,inf
2,125.0,82.0,23500.0,11595.0,112749.0,901.992000
4,0.0,0.0,23600.0,4575.0,19609.0,inf
6,7.0,3.0,23700.0,8865.0,25669.0,3667.000000
8,247.0,93.0,23800.0,10491.0,50801.0,205.672065
10,145.0,3.0,23900.0,10798.0,24505.0,169.000000
12,8804.0,3532.0,24000.0,9877.0,151669.0,17.227283
14,51.0,9.0,24100.0,2952.0,22267.0,436.607843
16,42.0,8.0,24200.0,4133.0,48645.0,1158.214286
18,77.0,15.0,24300.0,4559.0,41865.0,543.701299


In [47]:
3 //2

1

In [48]:
def topn(df,num):
    above_below = num//2 * 100
    start = nifty_price - above_below
    end = nifty_price + above_below
    return df[df['strike'].between(start,end)]

In [40]:
start = nifty_price - 300
end = nifty_price + 300
final_df = df[df['strike'].between(start,end)]

In [41]:
final_df

,call oi,CHNG IN OI,strike,CHNG IN OI.1,put oi,pcr
26,2611.0,123.0,24700.0,33739.0,104723.0,40.108388
28,4470.0,71.0,24800.0,9614.0,97138.0,21.731096
30,7648.0,4803.0,24900.0,3068.0,88166.0,11.527981
32,52850.0,21898.0,25000.0,36551.0,118164.0,2.235837
34,103768.0,72092.0,25100.0,3217.0,76524.0,0.737453
36,145127.0,94965.0,25200.0,63209.0,51803.0,0.356949
38,203144.0,113833.0,25300.0,28356.0,67116.0,0.330386


In [42]:
nifty_price

25000.0

In [44]:
top7 = topn(df,7)[['call oi',"put oi"]].sum()

In [69]:
def oi_analysis(df):
    for i in [7,5,3]:
        data = topn(df,i)
        print(i)
        print(pd.DataFrame(topn(df,i)[['call oi',"put oi"]].sum()).rename({0:i}))
        print()

In [70]:
oi_analysis(df)

7
                0
call oi  519618.0
put oi   603634.0

5
                0
call oi  313863.0
put oi   431795.0

3
                0
call oi  164266.0
put oi   282854.0



In [55]:
a = pd.DataFrame(top7).T.rename({0:7})

In [56]:
topn(df,7)

,call oi,CHNG IN OI,strike,CHNG IN OI.1,put oi,pcr
26,2611.0,123.0,24700.0,33739.0,104723.0,40.108388
28,4470.0,71.0,24800.0,9614.0,97138.0,21.731096
30,7648.0,4803.0,24900.0,3068.0,88166.0,11.527981
32,52850.0,21898.0,25000.0,36551.0,118164.0,2.235837
34,103768.0,72092.0,25100.0,3217.0,76524.0,0.737453
36,145127.0,94965.0,25200.0,63209.0,51803.0,0.356949
38,203144.0,113833.0,25300.0,28356.0,67116.0,0.330386


In [50]:
topn(df,5)

,call oi,CHNG IN OI,strike,CHNG IN OI.1,put oi,pcr
28,4470.0,71.0,24800.0,9614.0,97138.0,21.731096
30,7648.0,4803.0,24900.0,3068.0,88166.0,11.527981
32,52850.0,21898.0,25000.0,36551.0,118164.0,2.235837
34,103768.0,72092.0,25100.0,3217.0,76524.0,0.737453
36,145127.0,94965.0,25200.0,63209.0,51803.0,0.356949
